In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import wandb

In [3]:
import sys
sys.path.append("..")
from registration_tree import Rect, QuadTree

In [4]:
def add_help_fields(frame):
    
    frame["image_name_stem"] = [Path(image_name).stem for image_name in frame["image_name"]]    
    frame["patient_id"] = [name.split("_")[2] for name in frame["image_name"]]

    frame["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in frame["vector"]]
    frame["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in frame["vector"]]

    frame["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in frame["vector"]]
    frame["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in frame["vector"]]

    frame["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    frame["center"] = [np.array((center_x, center_y)) for center_x, center_y in zip(frame["center_x"], frame["center_y"])]

    frame["anno_width"] = [x2-x1 for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["anno_height"]= [y2-y1 for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    return frame

In [5]:
folder = Path("..")

slide_folder = Path("D:/Datasets/ScannerStudy")
if slide_folder.exists() == False:
    slide_folder = Path("/data/ScannerStudy")
if slide_folder.exists() == False:
    slide_folder = Path("/mnt/d/Datasets/ScannerStudy")
if slide_folder.exists() == False:
    slide_folder = Path("/data/ScannerStudy")
    
slide_files = {path.name: path for path in slide_folder.glob("*/*/*.*")}

In [6]:
def train(config=None):
    
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        annotations = add_help_fields(pd.read_csv(folder / "Validation/GT.csv"))
        annotations = annotations[annotations["image_type"] == config.image_type]
        
        source_scanner_annotations = annotations[annotations["scanner"] == config.source_scanner]
    
        dist_list, mean_reg_error_list = [], []
                        
        step = 0
        for patient_id in tqdm(source_scanner_annotations["patient_id"].unique()):

            source_annos = source_scanner_annotations[source_scanner_annotations["patient_id"] == patient_id]
            source_anno = source_annos.iloc[0]

            target_patient_annotations = annotations[annotations["patient_id"] == patient_id]

            for target_image_name in tqdm(target_patient_annotations["image_name"].unique()):
                image_dist_list = []

                target_annos = target_patient_annotations[target_patient_annotations["image_name"] == target_image_name]
                target_anno = target_annos.iloc[0]
                
                if source_anno.scanner == target_anno.scanner:
                    continue

                source_slide = openslide.OpenSlide(str(slide_files[source_anno.image_name]))
                target_slide = openslide.OpenSlide(str(slide_files[target_anno.image_name]))

                source_dimension = Rect.create(Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
                target_dimension = Rect.create(Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])


                qtree = QuadTree(source_dimension, source_slide, target_dimension, target_slide, debug=False, **config)
                                  
                mean_reg_error_list.append(qtree.mean_reg_error)
                
                intersections = list(set(source_annos["type_name"]).intersection(target_annos["type_name"]))
                
                for type_name in intersections:

                    source_anno = source_annos[source_annos["type_name"] == type_name].iloc[0]
                    target_anno = target_annos[target_annos["type_name"] == type_name].iloc[0]

                    box = [source_anno.center_x, source_anno.center_y, source_anno.anno_width, source_anno.anno_height]
                    target_box = [target_anno.center_x, target_anno.center_y, target_anno.anno_width, target_anno.anno_height]

                    trans_box = qtree.transform_boxes(np.array([box]))[0]

                    distance = np.linalg.norm(target_box[:2]-trans_box[:2])

                    dist_list.append(distance)
                    image_dist_list.append(distance)
                    
                image_dist_list = np.array(image_dist_list)
                wandb.log({
                    "dist_mean_image": image_dist_list.mean(),
                    "dist_mean_image": image_dist_list.min(),
                    "dist_mean_image": image_dist_list.max(),
                    "mean_reg_error_image": qtree.mean_reg_error,
                    "step": step,
                })
                
                step += 1
        
        dist_list, mean_reg_error_list = np.array(dist_list), np.array(mean_reg_error_list),
        
        wandb.log({
            "dist_mean": dist_list.mean(),
            "dist_min": dist_list.min(),
            "dist_max": dist_list.max(),
            
            "mean_reg_error": mean_reg_error_list.mean(),
        })

In [7]:
sweep_id = "82pnntx8"

In [8]:
wandb.agent(sweep_id, train, project="quadtree")

INFO - 2021-01-25 22:33:51,106 - pyagent - Starting sweep agent: entity=None, project=quadtree, count=None
wandb: Agent Starting Run: b5ab6sa6 with config:
wandb: 	crossCheck: False
wandb: 	filter_outliner: False
wandb: 	flann: True
wandb: 	homography: True
wandb: 	image_type: CCMCT
wandb: 	maxFeatures: 1024
wandb: 	point_extractor: sift
wandb: 	ratio: 0.6
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [1024, 1024]
wandb: 	use_gray: True
wandb: Currently logged in as: christianml (use `wandb login --relogin` to force relogin)


100%|██████████| 5/5 [00:19<00:00,  3.98s/it]


dist_mean_image,38.07222
step,19
_step,20
_runtime,21
_timestamp,1611610452
dist_mean,45.84855
dist_min,0.0
dist_max,333.32775
q_mean,-1.0
q_min,-1
q_max,-1


dist_mean_image,▃▁▂▄▃▁▃▅▅▁▄▃▃▁▃▄▂▁█▂
step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
_runtime,▁▁▂▂▃▃▄▄▄▅▅▅▅▆▆▇▇▇███
_timestamp,▁▁▂▂▃▃▄▄▄▅▅▅▅▆▆▇▇▇███
dist_mean,▁
dist_min,▁
dist_max,▁
q_mean,▁
q_min,▁
q_max,▁


wandb: Agent Starting Run: 5oedepsd with config:
wandb: 	crossCheck: False
wandb: 	filter_outliner: False
wandb: 	flann: True
wandb: 	homography: True
wandb: 	image_type: CCMCT
wandb: 	maxFeatures: 1024
wandb: 	point_extractor: sift
wandb: 	ratio: 0.4
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [1024, 1024]
wandb: 	use_gray: False


...

100%|██████████| 5/5 [00:51<00:00, 10.30s/it]


dist_mean_image,39.67029
step,19
_step,20
_runtime,52
_timestamp,1611646471
dist_mean,29.00276
dist_min,0.0
dist_max,251.91406
q_mean,-1.0
q_min,-1
q_max,-1


dist_mean_image,▂▁▂▄▄▁▅▂▂▁▃▃▂▁▂▃▃▁█▂
step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
_runtime,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▇▇███
_timestamp,▁▁▂▂▃▃▄▄▄▅▅▅▆▆▆▆▇▇███
dist_mean,▁
dist_min,▁
dist_max,▁
q_mean,▁
q_min,▁
q_max,▁


wandb: Agent Starting Run: em4yfrq6 with config:
wandb: 	crossCheck: False
wandb: 	filter_outliner: False
wandb: 	flann: True
wandb: 	homography: True
wandb: 	image_type: CCMCT
wandb: 	maxFeatures: 1024
wandb: 	point_extractor: sift
wandb: 	ratio: 0.5
wandb: 	source_scanner: Aperio
wandb: 	target_depth: 0
wandb: 	thumbnail_size: [8192, 8192]
wandb: 	use_gray: False


 25%|██▌       | 1/4 [02:27<07:21, 147.16s/it]wandb: Ctrl + C detected. Stopping sweep.
